In [ ]:
"""
Cell For Papermill Parameters
"""

labels_file = "../data/TCIA Biopsy Data_2020-07-14.xlsx"
features_file = "patients.csv"
out_file = "to_ngboost.csv"
model_scores = 'dnn_model_scores-MRI.csv'
model_scores2 = 'dnn_model_scores-US.csv'

In [ ]:
from IPython.display import Image
import pydicom as dicom
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import csv
import math
import glob
from pathlib import Path
import numpy as np

In [ ]:
#Load Features
df_in = pd.read_csv(features_file)

In [ ]:
#Load Labels
labels_df = pd.read_excel (labels_file)

In [ ]:
# PSA
psa_df = labels_df[['Patient Number','PSA (ng/mL)']]
psa_df = psa_df.drop_duplicates()

In [ ]:
labels_df = labels_df[['% Cancer in Core','Patient Number']]

In [ ]:
patients_df = labels_df[labels_df['% Cancer in Core'] > 0]

patient_mean_df = patients_df.groupby('Patient Number').mean()
patient_mean_df = patient_mean_df.reset_index()
patient_mean_df.rename(columns={"% Cancer in Core": "cancer_in_core_mean"},inplace=True)

patient_max_df = patients_df.groupby('Patient Number').max()
patient_max_df = patient_max_df.reset_index()
patient_max_df.rename(columns={"% Cancer in Core": "cancer_in_core_max"},inplace=True)

In [ ]:
labels_df = labels_df[labels_df['% Cancer in Core'] > 0]
labels_df = labels_df[['Patient Number']]
labels_df = labels_df.drop_duplicates()
labels_df["label_clas"] = 1

In [ ]:
labels_df = pd.merge(labels_df,patient_max_df,how='inner',left_on='Patient Number',right_on='Patient Number')
labels_df = pd.merge(labels_df,patient_mean_df,how='inner',left_on='Patient Number',right_on='Patient Number')

# JOIN DATA

In [ ]:
final_df = pd.merge(df_in,labels_df,how='left',left_on='name',right_on='Patient Number')
final_df["label_clas"] = final_df["label_clas"].fillna("0")
final_df["cancer_in_core_max"] = final_df["cancer_in_core_max"].fillna("0")
final_df["cancer_in_core_mean"] = final_df["cancer_in_core_mean"].fillna("0")
final_df["age"] = final_df["age"].str.replace('Y','')
final_df

# Load DNN Results MRI

In [ ]:
#Load Features
dnn_df = pd.read_csv(model_scores)
dnn_df = dnn_df[['patient','predicted','prob']]
dnn_df = dnn_df.drop_duplicates()
dnn_df = dnn_df.add_suffix('_MRI')
dnn_df

In [ ]:
dnn_df = dnn_df.groupby('patient_MRI').mean()
dnn_df = dnn_df.reset_index()
dnn_df

In [ ]:
final_df = pd.merge(final_df,dnn_df,how='left',left_on='name',right_on='patient_MRI')
final_df

# Load Models Scores US

In [ ]:
#Load Features
dnn_df = pd.read_csv(model_scores2)
dnn_df = dnn_df[['patient','predicted','prob']]
dnn_df = dnn_df.drop_duplicates()
dnn_df = dnn_df.add_suffix('_US')
dnn_df

In [ ]:
dnn_df = dnn_df.groupby('patient_US').mean()
dnn_df = dnn_df.reset_index()
dnn_df

In [ ]:
final_df = pd.merge(final_df,dnn_df,how='left',left_on='name',right_on='patient_US')
final_df

# ADD PSA

In [ ]:
psa_df
psa_df = psa_df.groupby('Patient Number').mean()
psa_df = psa_df.reset_index()
final_df = pd.merge(final_df,psa_df,how='left',left_on='name',right_on='Patient Number')

In [ ]:
final_df.head(5)

In [ ]:
final_df.rename(columns={"PSA (ng/mL)": "PSA"},inplace=True)
final_df.rename(columns={"predicted_MRI": "Model-MRI-DNN"},inplace=True)
final_df.rename(columns={"predicted_US": "Model-US-DNN"},inplace=True)

In [ ]:
final_df = final_df[["name","age","size","weight","ethnic_grp","occupation","smoking_status","label_clas","Model-MRI-DNN","prob_MRI","Model-US-DNN","prob_US","PSA"]]
final_df.head(5)

# Save Final Output

In [ ]:
final_df.to_csv(out_file, sep=',', encoding='utf-8', index=False)